### Evaluation of Semantic Segmentation Accuracy

In [ ]:
import os
import numpy as np
import pandas as pd

from pointtree.evaluation import semantic_segmentation_metrics
from pointtree.io import PointCloudReader

#### Point cloud files

In [ ]:
base_dir = '<insert path>'

datasets = {
    'TreeML': {
        '2023-01-09_tum_campus': {
            'file_path': '2023-01-09_tum_campus.laz',
            'street': '2023-01-09\_tum\_campus',
            'part': '',
            'subset': 'test',
            'short': 'TTC'
        },
        '2023-01-12_57': {
            'file_path': '2023-01-12_57.laz',
            'street': '2023-01-12\_57',
            'part': '',
            'subset': 'test',
            'short': 'T12\_57'
        },
        '2023-01-12_58': {
            'file_path': '2023-01-12_58.laz',
            'street': '2023-01-12\_58',
            'part': '',
            'subset': 'test',
            'short': 'T12\_58'
        },
        '2023-01-16_12': {
            'file_path': '2023-01-16_12.laz',
            'street': '2023-01-16\_12',
            'part': '',
            'subset': 'test',
            'short': 'T16\_12'
        },
        '2023-01-16_43': {
            'file_path': '2023-01-16_43.laz',
            'street': '2023-01-16\_43',
            'part': '',
            'subset': 'test',
            'short': 'T16\_43'
        },
    },
    'Essen': {
        'altendorfer_part_1': {
            'file_path': 'Altendorfer_p1_min_1.laz',
            'street': 'Altendorfer Straße',
            'part': 'part 1',
            'subset': 'val',
            'short': 'EAD1'
        },
        'altendorfer_part_2': {
            'file_path': 'Altendorfer_p2_min_1.laz',
            'street': 'Altendorfer Straße',
            'part': 'part 2',
            'subset': 'val',
            'short': 'EAD2'
        },
        'altenessener_part_4': {
            'file_path': 'Essen3_p2_min_1.laz',
            'street': 'Altenessener Straße',
            'part': 'part 4',
            'subset': 'val',
            'short': 'EAE4'
        },
        'altenessener_part_5': {
            'file_path': 'Essen3_p3_min_1.laz',
            'street': 'Altenessener Straße',
            'part': 'part 5',
            'subset': 'test',
            'short': 'EAE5'
        }
    },
    'Hamburg': {
        'armgart_straße_part_1': {
            'file_path': '000274_v2_min_1.laz',
            'street': 'Armgartstraße',
            'part': 'part 1',
            'subset': 'val',
            'short': 'HAG1'
        },
        'armgart_straße_part_2': {
            'file_path': '000275_000276_min_1.laz',
            'street': 'Armgartstraße',
            'part': 'part 2',
            'subset': 'test',
            'short': 'HAG2'
        }
    }
}

#### Calculate semantic segmentation metrics for each point cloud file

In [ ]:
reader = PointCloudReader()

for dataset in datasets:
    for file_id, file_infos in datasets[dataset].items():
        print("Process", file_id)
        metrics_folder = os.path.join(base_dir, 'Metrics', dataset, file_id)
        os.makedirs(metrics_folder, exist_ok=True)

        point_cloud = reader.read(os.path.join(base_dir, 'Data', dataset, '3_semantic_segmentation_processed',
                                               file_infos['file_path'])).data
    
        class_map = {
            "other": 0,
            "tree_trunk": 1,
            "tree_crown": 2,
            "tree_branch": 3,
            "low_vegetation": 4
        }

        aggregate_classes = {"tree": [1, 2, 3]}
        if dataset == "TreeML":
            aggregate_classes["other"] = [0, 4]

        metrics = semantic_segmentation_metrics(point_cloud['classification_target'].to_numpy(),
                                                point_cloud["classification_prediction"].to_numpy(),
                                                class_map, aggregate_classes=aggregate_classes)
        metrics["Dataset"] = dataset
        metrics["FileID"] = file_id
        metrics["Street"] = file_infos['street']
        metrics["Part"] = file_infos['part']
        metrics["Subset"] = file_infos['subset']
        metrics["Short"] = file_infos['short']
        metrics["Points"] = len(point_cloud)
        metrics["TreePoints"] = len(point_cloud[point_cloud["classification_target"].isin([1, 2, 3])])
        instance_ids = point_cloud[np.logical_and(point_cloud["instance_id"] != -1,
                                                 point_cloud["classification_target"].isin([1, 2, 3]))]["instance_id"]
        metrics["Trees"] = len(instance_ids.unique())

        complete_trees = 0
        for instance_id in instance_ids.unique():
            instance_trunk_points = point_cloud[np.logical_and(point_cloud["classification_target"] == 1,
                                                               point_cloud["instance_id"] == instance_id)]
            instance_crown_points = point_cloud[np.logical_and(point_cloud["classification_target"] == 2,
                                                               point_cloud["instance_id"] == instance_id)]
            if len(instance_trunk_points) > 0 and len(instance_crown_points) > 0:
                complete_trees += 1

        metrics["CompleteTrees"] = complete_trees

        metrics = pd.DataFrame([metrics])
    
        file_name = ".".join(file_infos['file_path'].split(".")[:-1]) + "_semantic_segmentation.csv"
        metrics.to_csv(os.path.join(metrics_folder, file_name), index=False)

####  Aggregate metrics for all point cloud files

In [ ]:
output_decimals = 3

metrics = []
for dataset in datasets:
    for file_id, file_infos in datasets[dataset].items():
        metrics_folder = os.path.join(base_dir, 'Metrics', dataset, file_id)
        file_name = ".".join(file_infos['file_path'].split(".")[:-1]) + "_semantic_segmentation.csv"
        semantic_segmentation_metrics_file = os.path.join(metrics_folder, file_name)
        if os.path.exists(semantic_segmentation_metrics_file):
            metrics.append(pd.read_csv(semantic_segmentation_metrics_file))

metrics = pd.concat(metrics)
metrics = metrics.rename({"m_iou": "mIoU",
                          "m_iou_aggregated": "mIoUAggr",
                          "tree_iou": "TreeIoU",
                          "tree_trunk_iou": "TrunkIoU",
                          "tree_branch_iou": "BranchIoU",
                          "tree_crown_iou": "CrownIoU",
                          "low_vegetation_iou": "LowVegetationIoU",
                          "other_iou": "OtherIoU"}, axis=1)
metric_columns = ["mIoU", "mIoUAggr", "TreeIoU", "TrunkIoU", "BranchIoU", "CrownIoU", "LowVegetationIoU", "OtherIoU"]
for metric_column in metric_columns:
    metrics[metric_column] = np.round(metrics[metric_column].to_numpy(), output_decimals)
metrics = metrics[["Dataset", "Street", "Part", "Subset", "Short", "Points", "TreePoints", "Trees", "CompleteTrees", *metric_columns]]
metrics.to_csv(os.path.join(base_dir, 'Metrics', f'dataset_semantic_segmentation_metrics.csv'), index=False)